In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf

from PIL import Image

In [ ]:
# 프레임 추출
base_dir = '../data/test_set' # 데이터 있는 폴더
file_list = os.listdir(base_dir) # base_dir 안에 있는 파일목록

pre_dir = os.path.join(base_dir, f'test') # 전처리한 사진 넣을 폴더 경로
os.makedirs(pre_dir, exist_ok=True) # 전처리한 사진 넣을 폴더 생성

for file in file_list:
    os.makedirs(os.path.join(pre_dir, file), exist_ok=True)
    vidcap = cv2.VideoCapture(os.path.join(base_dir, file))
    success,image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite(os.path.join(pre_dir, f'{file}/{count}.jpg'), image) # 전처리한 사진 저장    
        success,image = vidcap.read()
        count += 1

In [ ]:
# 데이터 전처리
test_dir = '../data/test_set/test'
folder_list = os.listdir(test_dir)
for folder in folder_list:
    folder_dir = os.path.join(test_dir, folder) # 작업중인 폴더 경로 ex) ../data/17-front
    pre_dir = os.path.join(base_dir, f'pre/{folder}') # 전처리한 사진 넣을 폴더 경로
    os.makedirs(pre_dir, exist_ok=True) # 전처리한 사진 넣을 폴더 생성
    file_list = os.listdir(folder_dir) # 폴더안의 사진 폴더 리스트
    count = 0
    for frame in range(16):
        img = Image.open(os.path.join(folder_dir, str(int(5+(frame*((len(file_list)-10)/16)))))+'.jpg')
        w, h = img.size
        x = (w - h) // 2 
        n = h//10

        # 이미지 잘라내기
        crop_img = img.crop((x + n, 0 + n, w - x - n, h - n))

        # 이미지 크기 변경
        crop_img = crop_img.resize(size=(112, 112)) 

        # BRG > RGB 변환
        crop_img = np.array(crop_img)
        crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)

        # 저장
        cv2.imwrite(f'../data/test_set/pre/{folder}/{count}.jpg', crop_img)
        count += 1

In [ ]:
# 모델 불러오기
model = tf.keras.models.load_model('../data/models/모델5_7.h5')

# 데이터 불러오기
test_set_dir = '../data/test_set/pre'
test_list = os.listdir(test_set_dir)
y_pred = list()
test_x_data = list()

for folder in test_list:
    folder_dir = os.path.join(test_set_dir, folder)
    temp_data = list()
    for file in range(16):
        img = cv2.imread(os.path.join(folder_dir, str(file)+'.jpg'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = img.reshape(img.shape+(1,))
        # 정규화
        img = img/255
        temp_data.append(img)
    temp_data = np.array(temp_data)
    test_x_data.append(temp_data)

# 리스트인 batch_x_data를 ndarray로
test_x_data = np.array(test_x_data)

# 축변경
test_x_data = np.swapaxes(test_x_data,1,2)
test_x_data = np.swapaxes(test_x_data,2,3)

result = model.predict(test_x_data)
y_pred.append(np.argmax(result, axis=1))

In [ ]:
y_pred = np.array(y_pred)
print(y_pred.ravel())
y_true = np.array([0,14,21,12,19,11,22,23,24,2,3,3,4,6,7,10,13,16,17])
print(y_true.ravel())

result = np.equal(y_pred.ravel(), y_true.ravel())
print(np.sum(result)/len(result))